In [1]:
import json
import cv2
import numpy as np
import datetime

In [2]:
# ai format
# 0  right_shoulder
# 1  right_elbow
# 2  right_wrist
# 3  left_shoulder
# 4  left_elbow
# 5  left_wrist
# 6  right_hip
# 7  right_knee
# 8  right_ankle
# 9  left_hip
# 10 left_knee
# 11 left_ankle
# 12 head
# 13 neck
annotations = {}
# coco annotations info.
annotations["info"] = {
    "description": "customer dataset format convert to COCO format",
    "url": "http://cocodataset.org",
    "version": "1.0",
    "year": 2019,
    "contributor": "matt",
    "date_created": "2019/12/19"
}
# coco annotations licenses.
annotations["licenses"] = [{
    "url": "https://www.apache.org/licenses/LICENSE-2.0.html",
    "id": 1,
    "name": "Apache License 2.0"
}]
# coco categories 
annotations["categories"] = [{
    "supercategory": "person",
    "id": 1,"name": "person",
    "keypoints": ["right_shoulder","right_elbow","right_wrist",
                  "left_shoulder", "left_elbow", "left_wrist", 
                  "right_hip", "right_knee", "right_ankle",
                  "left_hip", "left_knee", "left_ankle",
                 ],
    "skeleton": [[1,2],[2,3],[4,5],[5,6],[7,8],[8,9],[10,11],[11,12],
                 [1,4],[7,10]]
}]
annotations["images"] = []
annotations['annotations'] = []

In [3]:
json_file = '/media/hsw/E/datasets/ai-challenger/ai_test/ai_val/keypoint_validation_annotations_20170911.json'
f = open(json_file, encoding='utf-8')
annos = json.load(f)

In [4]:
img_path = '/media/hsw/E/datasets/ai-challenger/ai_test/ai_val/val'
img_id = 0
ann_id = 0
coco_pred_annos = []
for ann in annos:
    img_id += 1
#     print (ann)
    img = cv2.imread(img_path + "/" + ann['image_id'] + ".jpg")
    annotations["images"].append({
        "license": 1,
        "file_name": ann['image_id'] + ".jpg",
        "coco_url": ann['url'] if 'url' in ann.keys() else '',
        "height": img.shape[0],
        "width": img.shape[1],
        "data_captured": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "flickr_url": ann['url'] if 'url' in ann.keys() else '',
        "id": img_id
    })
    for key, value in ann['human_annotations'].items():
        ann_id += 1
        x1y1x2y2 = value
        kps = ann['keypoint_annotations'][key]
        nkps = 0
        for i in range(12):
            if kps[3*i+2] == 1:
                kps[3*i+2] = 2
                nkps += 1
            elif kps[3*i+2] == 2:
                kps[3*i+2] = 1
                nkps += 1
            elif kps[3*i+2] == 3:
                kps[3*i+2] = 0
            else:
                print ('error in kps')
        kps = kps[:12*3]
        annotations["annotations"].append({
            "id": ann_id,
            "image_id": img_id,
            "category_id": 1,
            "segmentation":[[value[0], value[1], value[2], value[1], value[2], value[3], value[0], value[3]]],
            "area": (value[2]-value[0]) * (value[3]-value[1]),
            "bbox": [value[0], value[1], value[2]-value[0], value[3]-value[1]],
            "iscrowd": 0,
            "keypoints":kps,
            "num_keypoints":nkps
                            
        })
        coco_pred_annos.append({
            "image_id": img_id,
            "category_id": 1,
            "keypoints": kps,
            "score": 0.9
        })
        if img_id % 10000 == 0:
            print (img_id)
#     break

10000
10000
10000
20000
20000
20000
30000
30000
30000
30000
30000


In [5]:
with open('/media/hsw/E/datasets/multipose_with_only_12_body_joints/aival.json', 'w') as fw:
    json.dump(annotations, fw)

In [6]:
with open('/media/hsw/E/datasets/cocoapi/annotations/ai_val_pred.json', 'w') as fw:
    json.dump(coco_pred_annos, fw)